In [20]:
import cv2
import time
import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# import matplotlib.pyplot as plt
# from imageai import Detection

In [2]:
from tensorflow import keras

In [3]:
class FPS:
    def __init__(self):
        self.pTime = time.time()
    def update(self, img=None, pos=(20, 50), color=(255, 0, 0), scale=3, thickness=3):
        cTime = time.time()
        try:
            fps = 1 / (cTime - self.pTime)
            self.pTime = cTime
            if img is None:
                return fps
            else:
                cv2.putText(img, f"FPS: {int(fps)}", pos, cv2.FONT_HERSHEY_PLAIN, scale, color, thickness)
                return fps, img
        except:
            return 0

In [4]:
cap = cv2.VideoCapture(0)
_, image = cap.read()
HEIGHT, WIDTH, _ = image.shape
cap.release()

In [5]:
classes = None
with open('./yolo/coco.names', 'rt') as f:
    classes = [line.strip() for line in f.readlines()]

In [6]:
configPath = './yolo/yolov3-tiny.cfg'
weightsPath = './yolo/yolov3-tiny.weights'

In [7]:
net = cv2.dnn_DetectionModel(weightsPath, configPath)

In [8]:
configPath2 = './yolo/cross-hands-yolov4-tiny.cfg'
weightsPath2 = './yolo/cross-hands-yolov4-tiny.weights'

In [9]:
net2 = cv2.dnn_DetectionModel(weightsPath2, configPath2)

In [10]:
net.setInputSize(width=WIDTH//3, height=HEIGHT//3)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

<dnn_Model 000001FDD89CB430>

In [11]:
net2.setInputSize(width=WIDTH//3, height=HEIGHT//3)
net2.setInputScale(1.0 / 127.5)
net2.setInputMean((127.5, 127.5, 127.5))
net2.setInputSwapRB(True)

<dnn_Model 000001FDD89CB6F0>

In [12]:
classifier = keras.models.load_model('./tfmodels/best_model.hdf5')

In [13]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 128)       36992     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 128)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 6, 6, 128)         0

In [27]:
fps = FPS()
cap = cv2.VideoCapture(0)

# def main():
while True:
    try:
        _,img = cap.read()
        fps.update(img)
        height, width, channels = img.shape

        # Detecting objects
        classIds, confs, bbox = net.detect(img, confThreshold = 0.5)
        handIds, handConfs, handBbox = net2.detect(img, confThreshold = 0.5)
        # print(classIds, bbox)

        bbox = list(bbox)
        confs = list(np.array(confs).reshape(1, -1)[0])
        handBbox = list(handBbox)
        handConfs = list(np.array(handConfs).reshape(1, -1)[0])
        # confs = list(map(float, confs))

        indices = cv2.dnn.NMSBoxes(bbox, confs, 0.5, 0.2)
        handindices = cv2.dnn.NMSBoxes(handBbox, handConfs, 0.5, 0.2)
        # print(indices)

        for i in handindices:
            box = handBbox[i]
            x, y, w, h = box[0], box[1], box[2], box[3]
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            handimg = img[box[1]:box[1]+box[3], box[0]:box[0]+box[2]]
            handimg = cv2.resize(handimg, (28, 28))
            handimg_gray = cv2.cvtColor(handimg, cv2.COLOR_BGR2GRAY)
            ans = classifier.predict(handimg_gray.reshape(1, 28, 28, 1))
            cv2.putText(img, f'hand gesture : {np.argmax(ans, axis=1)}', box[:2], cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            # cv2.imshow('hand', handimg)

        for i in indices:
            box = bbox[i]
            x, y, w, h = box[0], box[1], box[2], box[3]
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(img, f'{classes[classIds[i]].upper()} {confs[i]}', (box[0], box[1]), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

        cv2.imshow("Image", img)
        key = cv2.waitKey(1)
        if key == 27:
            break
    except Exception as e:
        print(e)
        break

# main()


cap.release()
cv2.destroyAllWindows()

In [15]:
from djitellopy import tello
from FaceTrackerModule import *


In [16]:
pErrorYV, pErrorUD = 0, 0
HEIGHT, WIDTH = 360, 480

drone = tello.Tello()
keepRecording = True

testing = 0

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.


In [17]:
def init():
    drone.connect()
    drone.streamon()
    if drone.get_battery() < 20:
        return False

    print(drone.get_battery())

    if not testing:
        drone.takeoff()
        drone.move_up(80)

    return True

In [18]:
def get_image():
    img = drone.get_frame_read().frame
    img = cv2.resize(img, (WIDTH, HEIGHT))
    return img

In [25]:
def main():
    
    if not init():
        return

    global pErrorYV, pErrorUD

    video1 = cv2.VideoWriter(os.path.join('images', '{}raw.avi'.format(time())), cv2.VideoWriter_fourcc(*'XVID'), 30, (WIDTH, HEIGHT))
    video2 = cv2.VideoWriter(os.path.join('images', '{}.avi'.format(time())), cv2.VideoWriter_fourcc(*'XVID'), 30, (WIDTH, HEIGHT))

    run = True
    while run:

        img = get_image()
        if img is None:
            continue

        raw = img.copy()

        img, face_info, hand_info = processFrame(img)
        commands, pErrorYV, pErrorUD = follow_face(face_info, pErrorYV, pErrorUD)
        displayText(img, commands, face_info, hand_info)    
        cv2.imshow('img', raw)
        cv2.imshow('processed', img)
        video1.write(raw)
        video2.write(img)

        lr, fb, ud, yv = commands
        if not testing:
            drone.send_rc_control(lr, fb, ud, yv)

        k = cv2.waitKey(30) & 0xff
        if k == 27:
            run = False

        if drone.get_battery() < 20:
            run = False

    video1.release()
    video2.release()

    cv2.destroyAllWindows()
    if not testing:
        drone.land()
    drone.end()